In [3]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
    list_strings = []
    for index in range(len(df)):
        #print(index)
        text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
        string = -1
        if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
            string = df.loc[index,"extended_tweet"]
        if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
            string = df.loc[index,"retweeted_status"]
        if type(string) == str :
            if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
                s = re.search('full_text\':(.+?)https',string).group(1)
            if(re.search('text\':(.+?)https',string)!= None):
                s = re.search('text\':(.+?)https',string).group(1)
            else:  #here then one should remove the http
                if (re.search('full_text\':\s\'(.+?)\'',string)!=None) :
                    s=re.search('full_text\':\s\'(.+?)\'',string).group(1)  
                if (re.search('\{\\\'full_text\\\':\s(.+?)\"',string)!=None) : 
                    re.search('\{\\\'full_text\\\':\s(.+?)\"',string).group(1)
            
            list_strings.append(s)
            #print(s)         
        else:
            list_strings.append(text)
            print(text)
      

    return list_strings

In [6]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [116]:
period = '_SeptOct2020'  # '', '_JanFeb2020', '_MarchApril2021', '_SeptOct2020'

In [117]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [118]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  1291
USA:  7436
China&USA:  8727


In [139]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

Output streaming troncato alle ultime 5000 righe.
Bars are closing in Paris as France places the city on maximum COVID-19 alert https://t.co/2hqt1RlQ5n https://t.co/qRcERjjjYZ
'A matter of survival': Lebanon's hospitals on the brink as COVID cases surge https://t.co/IKQ3YScaTd https://t.co/dI6xWNF9gd
'A blessing in disguise': COVID gives boost to Portugal's bicycle makers https://t.co/3nY5KHGXdE https://t.co/vadLdFUo7x
Russia calls for examining U.S. moves 'against the WHO' https://t.co/8wbY2Pvl1y https://t.co/YtTi8hxjrJ
U.S. calls for WHO reforms, timely information on outbreaks https://t.co/4piySkVAeM https://t.co/U02tTx6hvW
Set, miss, repeat: big brands and plastic recycling targets https://t.co/TILHUDRWVh https://t.co/ilwOylwPVn
Ireland faces resistance to 'nuclear' COVID-19 lockdown recommendation https://t.co/QmoBONNVey https://t.co/MhuDmXG0gE
U.S. coronavirus relief bill complicated by top Republicans testing positive https://t.co/NAGenlxJWU https://t.co/RusKyWCEMC
'Yikes!' UK's

In [140]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['daily case coronavirus infection high since mid summer see number expert express concern rapid acceleration covid case', 'presidential election approach voter see country deeply polarize key issue', 'government impose tough new restriction northern city late', 'covid case surpass million week pandemic take toll labor market economy', 'number coronavirus confirm case surpass one million week many health worker', 'number death africa coronavirus surpass mark world health say', 'become first country western report one million confirm covid infection continent grapple resurgence case', 'become first country western reach million confirm covid case accord health ministry', 'president urge vigilance possible second wave covid', 'germany federal minister health test positive coronavirus local medium report']

USA:
['global covid case surpass million coronavirus news follow coverage', 'since start covid pandemic lot misinformation look common coronavirus', 'myth shoe clothes spread co

In [141]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  2235
USA:  4587
China&USA:  4968


In [142]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['covid', 'case', 'coronavirus']

USA
['covid', 'coronavirus', 'say', 'trump', 'vaccine', 'new', 'case', 'test', 'president', 'pandemic', 'positive', 'report', 'house']

China&USA
['covid', 'coronavirus', 'say', 'trump', 'case', 'new', 'vaccine', 'test', 'president', 'report', 'pandemic', 'positive']


# Build Network

In [143]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    545
1          case    273
2   coronavirus    256
3           say    198
4           new    194
5         china    174
6        report    172
7          test    151
8     president    148
9        health    133
10     pandemic    124
11        trump    116
12      country    115
13      vaccine    114
14        watch    111
15     positive    102
16        world     92
17      million     90
18        death     85
19       accord     81
20       people     81
21          day     72
22       number     67
23         year     64
24    infection     63
25         city     62
26       global     61
27        first     60
28        house     58
29     national     56

USA
           Word  Count
0         covid   4105
1   coronavirus   2149
2           say   1475
3         trump   1265
4       vaccine   1121
5           new   1084
6          case   1012
7          test    987
8     president    974
9      pandemic    575
10     positive    565


In [144]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [61]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [167]:
network_China, network_df_China = create_network(cleaned_text_China)
network_USA, network_df_USA = create_network(cleaned_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_text_China_USA)

In [146]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                     weight
(new, case)             148
(report, case)          147
(case, covid)           141
(china, covid)          112
(test, positive)        109
(covid, say)            105
(covid, new)            105
(report, new)           104
(president, trump)       96
(covid, pandemic)        86
(report, covid)          84
(president, covid)       80
(case, health)           75
(covid, test)            74
(covid, trump)           64
(case, coronavirus)      64
(say, president)         61
(new, health)            61
(white, house)           59
(country, covid)         56
(new, coronavirus)       56
(covid, positive)        56
(trump, test)            55
(covid, million)         54
(case, million)          54
(case, number)           54
(case, total)            54
(death, case)            51
(case, accord)           50
(health, report)         50

USA:
                          weight
(president, trump)          1010
(say, covid)                 807
(covid, vaccine)    

#Graph


In [127]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [168]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [169]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  2235
Edges:  46727
Is connected:  False

USA:
Nodes:  4587
Edges:  178692
Is connected:  True

China&USA:
Nodes:  4968
Edges:  207328
Is connected:  True


# PageRank

In [149]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [26]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [27]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [150]:
thr = 0.0004
print('China: ', len(threshold(pr_China,thr)))
print()
print(threshold(pr_China,thr).iloc[:30])
print()
print('USA: ', len(threshold(pr_USA,thr)))
print()
print(threshold(pr_USA,thr).iloc[:30])
print()
print('China&USA: ', len(threshold(pr_China_USA,thr)))
print()
print(threshold(pr_China_USA,thr).iloc[:30])

China:  519

              0         1
0         covid  0.036667
1          case  0.018059
2   coronavirus  0.016841
3           say  0.015372
4         china  0.014078
5           new  0.013475
6        report  0.012269
7     president  0.011120
8          test  0.010213
9        health  0.009855
10     pandemic  0.008913
11      country  0.008691
12        trump  0.008686
13        world  0.007196
14     positive  0.007175
15      vaccine  0.007131
16      million  0.006109
17       accord  0.005896
18          day  0.005860
19        death  0.005740
20       people  0.005582
21         city  0.004985
22         year  0.004928
23    infection  0.004882
24       number  0.004776
25        first  0.004634
26        house  0.004509
27       global  0.004153
28         hour  0.003919
29     national  0.003898

USA:  513

              0         1
0         covid  0.038166
1   coronavirus  0.023319
2           say  0.018473
3         trump  0.017661
4     president  0.013832
5       vacci

# TF-IDF

In [151]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [152]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [153]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [154]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
watch          0.057550
covid          0.046426
case           0.037155
coronavirus    0.033979
new            0.028119
report         0.026203
update         0.025734
say            0.023554
test           0.022541
china          0.021576
health         0.020003
president      0.019756
vaccine        0.017704
trump          0.017490
pandemic       0.017101
country        0.016832
positive       0.016704
million        0.016502
death          0.016156
sept           0.015965
late           0.014840
accord         0.014146
world          0.013591
people         0.013359
number         0.012477
day            0.011692
global         0.011513
national       0.010918
infection      0.010908
year           0.010387
dtype: float64

USA:
covid          0.056660
coronavirus    0.039230
case           0.030680
new            0.028935
say            0.028032
vaccine        0.027544
test           0.024141
trump          0.023875
report         0.020494
president      0.019379
positive    

# reduced graph

In [155]:
# less important words:
less_important_words_China = list(threshold_reverse(pr_China,thr)[0])

less_important_words_USA = list(threshold_reverse(pr_USA,thr)[0])

less_important_words_China_USA = list(threshold_reverse(pr_China_USA,thr)[0])

In [73]:
def clean_words(cleaned_text, unuseful_words):
  re_cleaned_text = cleaned_text.copy()
  for txt in range(len(re_cleaned_text)):
    if txt % 2000 == 0:
      print(txt, '/',len(re_cleaned_text))
    w = re_cleaned_text[txt].split()
    for word in unuseful_words:
      while word in w:
        w.remove(word)
    re_cleaned_text[txt] = ' '.join(w)
  return re_cleaned_text

In [156]:
cleaned_mostimp_text_China = clean_words(cleaned_text_China,less_important_words_China)
cleaned_mostimp_text_USA = clean_words(cleaned_text_USA,less_important_words_USA)
cleaned_mostimp_text_China_USA = clean_words(cleaned_text_China_USA,less_important_words_China_USA)

0 / 1291
0 / 7436
2000 / 7436
4000 / 7436
6000 / 7436
0 / 8727
2000 / 8727
4000 / 8727
6000 / 8727
8000 / 8727


In [157]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  519
USA:  513
China&USA:  508


In [158]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    545
1          case    273
2   coronavirus    256
3           say    198
4           new    194
5         china    174
6        report    172
7          test    151
8     president    148
9        health    133
10     pandemic    124
11        trump    116
12      country    115
13      vaccine    114
14        watch    111
15     positive    102
16        world     92
17      million     90
18        death     85
19       accord     81
20       people     81
21          day     72
22       number     67
23         year     64
24    infection     63
25         city     62
26       global     61
27        first     60
28        house     58
29     national     56

USA
           Word  Count
0         covid   4105
1   coronavirus   2149
2           say   1475
3         trump   1265
4       vaccine   1121
5           new   1084
6          case   1012
7          test    987
8     president    974
9      pandemic    575
10     positive    565


In [159]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [160]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [161]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                     weight
(new, case)             148
(report, case)          147
(case, covid)           141
(china, covid)          112
(test, positive)        109
(covid, new)            105
(covid, say)            105
(report, new)           104
(president, trump)       96
(covid, pandemic)        86
(report, covid)          84
(president, covid)       80
(case, health)           75
(covid, test)            74
(case, coronavirus)      64
(covid, trump)           64
(say, president)         61
(new, health)            61
(white, house)           59
(country, covid)         56
(covid, positive)        56
(new, coronavirus)       56
(trump, test)            55
(covid, million)         54
(case, million)          54
(case, number)           54
(case, total)            54
(death, case)            51
(case, accord)           50
(confirm, case)          50

USA:
                          weight
(president, trump)          1010
(say, covid)                 807
(covid, vaccine)    

In [44]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [162]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [163]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  519
Edges:  19747
Is connected:  False

USA:
Nodes:  513
Edges:  48439
Is connected:  True

China&USA:
Nodes:  508
Edges:  52472
Is connected:  True


In [170]:
for c in nx.connected_components(G_China):
  print(c)

{'let', 'technician', 'condition', 'railway', 'earmark', 'list', 'athlete', 'match', 'feeling', 'dream', 'caution', 'clinical', 'consecutive', 'mark', 'benjamin', 'cancel', 'crush', 'business', 'comfortable', 'center', 'musician', 'include', 'survive', 'video', 'dining', 'find', 'breakdown', 'sudden', 'two', 'grip', 'strict', 'speaker', 'security', 'balcony', 'serious', 'gap', 'shutdown', 'age', 'infect', 'batch', 'preparation', 'traditional', 'france', 'xinjiang', 'legislative', 'routine', 'plunge', 'reunification', 'contract', 'pope', 'somewhere', 'beef', 'infectious', 'representative', 'global', 'lancet', 'soar', 'ought', 'resident', 'regulation', 'comic', 'tackle', 'pilgrimage', 'condemn', 'tap', 'build', 'interview', 'ground', 'forecast', 'sanctuary', 'bicycle', 'non', 'authorized', 'king', 'free', 'exercise', 'news', 'viking', 'integrity', 'pandemic', 'facebook', 'seek', 'expand', 'strain', 'range', 'epicenter', 'grow', 'cruise', 'australian', 'uncertainty', 'amid', 'ever', 'demo

In [171]:
cleaned_mostimp_text_China = clean_words(cleaned_mostimp_text_China,['exhibition','victory','talented','guitar','girl','exclusive'])

0 / 1291


In [172]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word
keys_China = freq_dict_China.keys()  
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
G_China = get_graph(network_China)

print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))

China:
Nodes:  513
Edges:  19740
Is connected:  True


#Save edge list

In [173]:
filename = './edgelist_China_SeptOct2020.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_SeptOct2020.csv
files.download("edgelist_China_SeptOct2020.csv")

filename = './edgelist_USA_SeptOct2020.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_SeptOct2020.csv
files.download("edgelist_USA_SeptOct2020.csv")

filename = './edgelist_China_USA_SeptOct2020.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_SeptOct2020.csv
files.download("edgelist_China_USA_SeptOct2020.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [48]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [174]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_SeptOct2020")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_SeptOct2020")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_USA_SeptOct2020")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
            Id        Label
0        covid        covid
1         case         case
2  coronavirus  coronavirus
3          say          say
4          new          new

USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2          say          say
3        trump        trump
4      vaccine      vaccine

China&USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2          say          say
3        trump        trump
4         case         case

